# Dia 1 - Importação de Dados

In [4]:
# Importando bibliotecas

!pip install pyarrow
!pip install pandas

In [4]:
import pandas as pd
import glob

### Criando repositórios para os arquivos

In [7]:
# Criando diretórios e pastas para receber os arquivos do GitHub

from pathlib import Path

# Cria um caminho (sem criar ainda no sistema de arquivos)
pasta_emprestimos = Path("dados/emprestimos")
pasta_exemplares = Path("dados/exemplares")

# Criação das pastas, incluindo as intermediárias se não existirem
pasta_emprestimos.mkdir(parents=True, exist_ok=True)
pasta_exemplares.mkdir(parents=True, exist_ok=True)

### Realizando a coleta de dados

In [11]:
# Baixando os arquivos para as pastas correspondentes

import requests

# Baixando CSVs

# Lista de anos para concatenar o nome do arquivo.
# Os arquivos CSV tem o mesmo prefixo mudando apenas o final.
# Para não importar os arquivos um a um, cria-se uma lista com o nome final,
# e concatena com o prefixo, realizando o download com um loop
lista_emprestimos = []

# Criando a lista automáticamente
for ano in range(2010, 2021):
    for semestre in [1, 2]:
        if ano == 2020 and semestre == 2:
            continue # pula o segundo semestre de 2020 e encerra o loop
        lista_emprestimos.append(f'{ano}{semestre}')

# Loop de download
for ano in lista_emprestimos:
    # passando o link para download e concatenando o ano ao final do nome do arquivo
    url = f"https://raw.githubusercontent.com/franciscofoz/7_Days_of_Code_Alura-Python-Pandas/main/Dia_1-Importando_dados/Datasets/dados_emprestimos/emprestimos-{ano}.csv"
    
    # faz uma requisição HTTP e armazena (get) a resposta em r
    r = requests.get(url)
    
    # armazenando o arquivo na pasta correspondente
    with open(f"dados/emprestimos/emprestimos-{ano}.csv", "wb") as f:
        f.write(r.content) # escreve o conteúdo de "r", que é o que ele pegou do link, o arquivo.

# Baixando o Parquet

parquet_url = f"https://github.com/franciscofoz/7_Days_of_Code_Alura-Python-Pandas/raw/main/Dia_1-Importando_dados/Datasets/dados_exemplares.parquet"
r = requests.get(parquet_url)

with open("dados/exemplares/dados_exemplares.parquet", "wb") as f:
    f.write(r.content)

In [12]:
# Criando uma lista com os caminhos de todos os arquivos .csv na pasta emprestimos
caminhos_csv = glob.glob("dados/emprestimos/*.csv")
print(caminhos_csv)

['dados/emprestimos\\emprestimos-20101.csv', 'dados/emprestimos\\emprestimos-20102.csv', 'dados/emprestimos\\emprestimos-20111.csv', 'dados/emprestimos\\emprestimos-20112.csv', 'dados/emprestimos\\emprestimos-20121.csv', 'dados/emprestimos\\emprestimos-20122.csv', 'dados/emprestimos\\emprestimos-20131.csv', 'dados/emprestimos\\emprestimos-20132.csv', 'dados/emprestimos\\emprestimos-20141.csv', 'dados/emprestimos\\emprestimos-20142.csv', 'dados/emprestimos\\emprestimos-20151.csv', 'dados/emprestimos\\emprestimos-20152.csv', 'dados/emprestimos\\emprestimos-20161.csv', 'dados/emprestimos\\emprestimos-20162.csv', 'dados/emprestimos\\emprestimos-20171.csv', 'dados/emprestimos\\emprestimos-20172.csv', 'dados/emprestimos\\emprestimos-20181.csv', 'dados/emprestimos\\emprestimos-20182.csv', 'dados/emprestimos\\emprestimos-20191.csv', 'dados/emprestimos\\emprestimos-20192.csv', 'dados/emprestimos\\emprestimos-20201.csv']


### Criando um DataFrame de DataFrames

In [13]:
# Adicionando todos os CSVs em um único DataFrame

dataframes = []

for caminho in caminhos_csv: 
    print(f'Lendo o arquivo: {caminho}') 
    try:
        df = pd.read_csv(caminho, sep = ',', encoding = 'utf-8') 
        print(f'Primeiras linhas do arquivo {caminho}:')
        print(df.head()) 
        dataframes.append(df)
    except Exception as e:
        print(f'Erro ao ler {caminho}: {e}')

if not dataframes:
    print(f'Nenhum arquivo foi lido corretamente!')
else:
    df_emprestimos = pd.concat(dataframes, ignore_index=True)
    print("DataFrame final concatenado:")
    print(df_emprestimos.head())

Lendo o arquivo: dados/emprestimos\emprestimos-20101.csv
Primeiras linhas do arquivo dados/emprestimos\emprestimos-20101.csv:
   id_emprestimo codigo_barras                 data_renovacao  \
0         709684       L095049                            NaN   
1         709685       L167050                            NaN   
2         709686    2006017618  2010/01/26 08:07:01.738000000   
3         709687       L184117  2010/01/18 11:07:46.470000000   
4         709684       L095049                            NaN   

                 data_emprestimo                 data_devolucao  \
0  2010/01/04 07:44:10.721000000  2010/01/05 16:26:12.662000000   
1  2010/01/04 07:44:10.750000000  2010/01/12 07:34:13.934000000   
2  2010/01/04 08:08:44.081000000  2010/02/25 07:36:25.800000000   
3  2010/01/04 08:24:21.284000000  2010/02/03 08:58:45.692000000   
4  2010/01/04 07:44:10.721000000  2010/01/05 16:26:12.662000000   

   matricula_ou_siape    tipo_vinculo_usuario  
0        2.008023e+09      ALUNO

### Concatenando os DataFrames

In [14]:
# Juntando todas as tabelas em uma única

df_emprestimos.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2.008023e+09,ALUNO DE GRADUAÇÃO
1,709685,L167050,NaN,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2.008023e+09,ALUNO DE GRADUAÇÃO
2,709686,2006017618,2010/01/26 08:07:01.738000000,2010/01/04 08:08:44.081000000,2010/02/25 07:36:25.800000000,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO
3,709687,L184117,2010/01/18 11:07:46.470000000,2010/01/04 08:24:21.284000000,2010/02/03 08:58:45.692000000,2.007211e+08,ALUNO DE GRADUAÇÃO
4,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2.008023e+09,ALUNO DE GRADUAÇÃO


In [15]:
df_emprestimos.columns

Index(['id_emprestimo', 'codigo_barras', 'data_renovacao', 'data_emprestimo',
       'data_devolucao', 'matricula_ou_siape', 'tipo_vinculo_usuario'],
      dtype='object')

### Verificando valores nulos

In [16]:
# Verificando e tratando valores nulos

df_emprestimos.isnull().sum()

id_emprestimo                 0
codigo_barras                 0
data_renovacao          1285720
data_emprestimo               0
data_devolucao             6471
matricula_ou_siape         3170
tipo_vinculo_usuario          0
dtype: int64

### Verificando Duplicatas

In [17]:
# Verificando duplicatas no DataFrame inteiro

df_emprestimos.duplicated().sum()

np.int64(37)

In [18]:
df_emprestimos.value_counts()

id_emprestimo  codigo_barras  data_renovacao                 data_emprestimo                data_devolucao                 matricula_ou_siape  tipo_vinculo_usuario  
709704         2006021442     2010/01/17 18:56:43.923000000  2010/01/04 09:58:46.522000000  2010/02/01 13:59:02.528000000  2.008010e+09        ALUNO DE GRADUAÇÃO        7
709705         2009032168     2010/01/17 18:56:43.909000000  2010/01/04 09:58:46.549000000  2010/01/20 13:14:34.464000000  2.008010e+09        ALUNO DE GRADUAÇÃO        7
709698         2009047725     2010/01/18 14:44:41.163000000  2010/01/04 09:21:19.099000000  2010/02/02 12:02:38.444000000  2.009047e+09        ALUNO DE GRADUAÇÃO        3
709686         2006017618     2010/01/26 08:07:01.738000000  2010/01/04 08:08:44.081000000  2010/02/25 07:36:25.800000000  2.008112e+09        ALUNO DE PÓS-GRADUAÇÃO    2
2461099        2012008348     2016/06/27 20:35:46.855000000  2016/06/14 11:20:33.254000000  2016/07/11 14:37:29.000000000  2.015022e+09        ALUNO D

In [19]:
# Mostra todas as linhas duplicadas

df_emprestimos[df_emprestimos.duplicated()]

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
4,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2.008023e+09,ALUNO DE GRADUAÇÃO
5,709685,L167050,NaN,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2.008023e+09,ALUNO DE GRADUAÇÃO
6,709686,2006017618,2010/01/26 08:07:01.738000000,2010/01/04 08:08:44.081000000,2010/02/25 07:36:25.800000000,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO
7,709687,L184117,2010/01/18 11:07:46.470000000,2010/01/04 08:24:21.284000000,2010/02/03 08:58:45.692000000,2.007211e+08,ALUNO DE GRADUAÇÃO
19,709698,2009047725,2010/01/18 14:44:41.163000000,2010/01/04 09:21:19.099000000,2010/02/02 12:02:38.444000000,2.009047e+09,ALUNO DE GRADUAÇÃO
20,709698,2009047725,2010/01/18 14:44:41.163000000,2010/01/04 09:21:19.099000000,2010/02/02 12:02:38.444000000,2.009047e+09,ALUNO DE GRADUAÇÃO
28,709704,2006021442,2010/01/17 18:56:43.923000000,2010/01/04 09:58:46.522000000,2010/02/01 13:59:02.528000000,2.008010e+09,ALUNO DE GRADUAÇÃO
29,709705,2009032168,2010/01/17 18:56:43.909000000,2010/01/04 09:58:46.549000000,2010/01/20 13:14:34.464000000,2.008010e+09,ALUNO DE GRADUAÇÃO
30,709704,2006021442,2010/01/17 18:56:43.923000000,2010/01/04 09:58:46.522000000,2010/02/01 13:59:02.528000000,2.008010e+09,ALUNO DE GRADUAÇÃO
31,709705,2009032168,2010/01/17 18:56:43.909000000,2010/01/04 09:58:46.549000000,2010/01/20 13:14:34.464000000,2.008010e+09,ALUNO DE GRADUAÇÃO


In [20]:
# Garantindo que as linhas são realmente duplicadas

df_emprestimos[df_emprestimos.duplicated(keep=False)]

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2.008023e+09,ALUNO DE GRADUAÇÃO
1,709685,L167050,NaN,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2.008023e+09,ALUNO DE GRADUAÇÃO
2,709686,2006017618,2010/01/26 08:07:01.738000000,2010/01/04 08:08:44.081000000,2010/02/25 07:36:25.800000000,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO
3,709687,L184117,2010/01/18 11:07:46.470000000,2010/01/04 08:24:21.284000000,2010/02/03 08:58:45.692000000,2.007211e+08,ALUNO DE GRADUAÇÃO
4,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2.008023e+09,ALUNO DE GRADUAÇÃO
...,...,...,...,...,...,...,...
1504570,2461095,2014007744,NaN,2016/06/14 11:16:34.459000000,2016/06/22 08:04:39.000000000,2.016013e+10,ALUNO DE GRADUAÇÃO
1504571,2461096,2016005538,NaN,2016/06/14 11:16:58.596000000,2016/07/04 16:07:41.000000000,2.016014e+10,ALUNO DE GRADUAÇÃO
1504572,2461097,2014083427,NaN,2016/06/14 11:17:29.152000000,2016/06/16 09:17:19.000000000,2.016014e+10,ALUNO DE GRADUAÇÃO
1504573,2461098,2014049460,NaN,2016/06/14 11:18:59.254000000,2016/06/26 13:44:13.000000000,2.017005e+10,ALUNO DE GRADUAÇÃO


### Excluindo duplicatas

In [21]:
df_emprestimos = df_emprestimos.drop_duplicates()
df_emprestimos.value_counts()

id_emprestimo  codigo_barras  data_renovacao                 data_emprestimo                data_devolucao                 matricula_ou_siape  tipo_vinculo_usuario  
709686         2006017618     2010/01/26 08:07:01.738000000  2010/01/04 08:08:44.081000000  2010/02/25 07:36:25.800000000  2.008112e+09        ALUNO DE PÓS-GRADUAÇÃO    1
2336122        2011033079     2016/11/21 02:01:58.488000000  2016/11/10 17:00:12.989000000  2016/12/02 14:21:49.000000000  2.016023e+09        ALUNO DE GRADUAÇÃO        1
2336103        2014085936     2016/11/25 19:25:36.215000000  2016/11/10 16:54:52.447000000  2016/12/13 14:30:52.000000000  2.014047e+09        ALUNO DE GRADUAÇÃO        1
2336104        2011027960     2016/11/23 17:27:54.642000000  2016/11/10 16:55:17.956000000  2016/11/28 10:32:38.000000000  2.016083e+09        ALUNO DE GRADUAÇÃO        1
2336105        2014000897     2016/12/02 15:02:53.356000000  2016/11/10 16:55:19.851000000  2016/12/23 13:53:23.000000000  2.014071e+09        ALUNO D

In [22]:
# Verificando duplicatas no DataFrame inteiro

df_emprestimos.duplicated().sum()

np.int64(0)

In [23]:
# Mostra todas as linhas duplicadas

df_emprestimos[df_emprestimos.duplicated()]

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario


### Últimos ajustes no DataFrame de empréstimos

In [24]:
# Verificando os tipos de dados do DataFrame

df_emprestimos.dtypes

id_emprestimo             int64
codigo_barras            object
data_renovacao           object
data_emprestimo          object
data_devolucao           object
matricula_ou_siape      float64
tipo_vinculo_usuario     object
dtype: object

In [25]:
# Verificando valores nulos na coluna código de barras, que será a relação com o arquivo dados do acervo

df_emprestimos['codigo_barras'].isnull().sum()

np.int64(0)

In [26]:
# Verificando padrão das colunas de datas

df_emprestimos['data_devolucao'] = pd.to_datetime(df['data_devolucao'], errors = 'coerce')
df_emprestimos['data_emprestimo'] = pd.to_datetime(df['data_emprestimo'], errors = 'coerce')
df_emprestimos['data_renovacao'] = pd.to_datetime(df['data_renovacao'], errors = 'coerce')

### Importando os dados do acervo

In [27]:
# Carregando o arquivo de dados do acervo

df_acervo = pd.read_parquet('dados/exemplares/dados_exemplares.parquet')
df_acervo.head()

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15


### Validações do DataFrame de acervo

In [28]:
# Verificações do arquivo de dados do acervo

df_acervo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 546237 entries, 0 to 568584
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id_exemplar       546237 non-null  int64 
 1   codigo_barras     546237 non-null  object
 2   colecao           546237 non-null  object
 3   biblioteca        546237 non-null  object
 4   status_material   546237 non-null  object
 5   localizacao       546237 non-null  int64 
 6   registro_sistema  546237 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 33.3+ MB


In [29]:
# Verifica valores nulos

df_acervo.isnull().sum()

id_exemplar         0
codigo_barras       0
colecao             0
biblioteca          0
status_material     0
localizacao         0
registro_sistema    0
dtype: int64

In [30]:
# verifica colunas únicas ou  irrelevantes

df_acervo.nunique()

id_exemplar         546141
codigo_barras       545049
colecao                 18
biblioteca              25
status_material          3
localizacao            900
registro_sistema    198503
dtype: int64

In [31]:
# Visualiza amostras aleatórias

df_acervo.sample(5)

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
240268,242951,2008012430,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,627,101243
246346,247743,2008038801,Acervo Circulante,Biblioteca Setorial do Centro de Ciências Huma...,REGULAR,643,104172
145006,156450,2007033014,Obras de Referência,Biblioteca Setorial do Centro Ciências da Saúd...,NÃO CIRCULA,632,58081
394036,1003160,2012036591,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,375,184319
203941,211116,2006034288,Dissertações,Biblioteca Central Zila Mamede,ESPECIAL,621,85022


In [32]:
# Verifica o tipo de dados

df_acervo.dtypes

id_exemplar          int64
codigo_barras       object
colecao             object
biblioteca          object
status_material     object
localizacao          int64
registro_sistema     int64
dtype: object

### Unindo os dois DataFrames

In [33]:
# Mesclando os DataFrames de Emprestimos e Acervo

# O método merge, irá mesclar os DataFrames, usando como referência a coluna 'codigo_barras'
df_completo = pd.merge(df_emprestimos, # DataFrame da esquerda
                       df_acervo, # DataFrame da direita
                       on = 'codigo_barras', # Coluna em comum entre os dois
                       how = 'left' # Tipo de junção (LEFT JOIN)
                       )
# how = left significa: "traga todos os registros do DataFrame da esquerda (df_emprestimos) e complemente com os dados do da direita (df_acervo) quando houver correspondência"

df_completo.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
0,709684,L095049,NaT,2020-01-02 08:17:30.290,2020-01-07 11:14:07,2.008023e+09,ALUNO DE GRADUAÇÃO,13259.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630.0,4225.0
1,709685,L167050,NaT,2020-01-02 08:17:30.391,2020-01-07 11:14:17,2.008023e+09,ALUNO DE GRADUAÇÃO,70865.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647.0,25009.0
2,709686,2006017618,NaT,2020-01-02 08:17:30.436,2020-01-07 11:24:46,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO,195347.0,Acervo Circulante,Biblioteca Setorial Prof. Rodolfo Helinski - E...,REGULAR,640.0,75019.0
3,709687,L184117,2020-01-17 09:56:18.380,2020-01-02 08:36:40.163,2020-01-31 14:53:52,2.007211e+08,ALUNO DE GRADUAÇÃO,131639.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,609.0,44494.0
4,709688,2007027500,2020-01-15 08:47:19.174,2020-01-02 09:51:24.156,2020-01-29 12:18:08,2.009046e+09,ALUNO DE GRADUAÇÃO,225646.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,686.0,91736.0


### Verificando se há linhas sem correspondência e o percentual

In [34]:
# Verificando quantas linhas não encontraram correspondência no acervo

faltando_acervo = df_completo['id_exemplar'].isnull().sum()
print(f'Linhas sem correspondência no acervo: {faltando_acervo}')

percent_faltando = (faltando_acervo / len(df_completo)) * 100
print(f'{percent_faltando:.2f}% dos empréstimos não têm dados de acervo.')

Linhas sem correspondência no acervo: 189257
8.37% dos empréstimos não têm dados de acervo.


### Verificando quais os códigos de barra que não encontraram correspondência

In [35]:
# Verificando quais códigos de barras não encontraram correspondência

# Seleciona a coluna 'id_exemplar' que vem de df_acervo. Se ela tiver NaN após o merge, não houve correspondência para 
# Aquele código de barras.
# isna() retorna True para valores NaN, e False para valores preenchidos
# df_sem_match está recebendo os emprestimos que não tiveram correspondência com os ID's de exemplares do acervo.
df_sem_match = df_completo[df_completo['id_exemplar'].isna()]
df_sem_match.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
17,709701,L163841,2020-01-15 17:06:52.273,2020-01-02 10:26:04.065,2020-01-31 10:57:32,2.008029e+09,ALUNO DE GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
98,709782,L165850,2020-01-17 10:46:58.963,2020-01-02 16:37:54.645,2020-02-27 18:20:56,2.006213e+08,ALUNO DE GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
109,709793,L178388,NaT,2020-01-02 17:04:53.243,2020-01-07 13:24:47,2.005063e+08,ALUNO DE GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
118,709802,2009032201,NaT,2020-01-02 17:51:52.860,2020-01-21 07:56:58,2.009054e+09,ALUNO DE GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
119,709803,L188583,NaT,2020-01-02 17:51:52.899,2020-01-21 07:57:06,2.010117e+09,ALUNO DE PÓS-GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Retorna os valores únicos da coluna código de barras
# Quais foram os códigos de barras que não foram encontrados no acervo?
df_sem_match['codigo_barras'].unique()

array(['L163841', 'L165850', 'L178388', ..., 'L143612', '2018007270',
       'Y021773'], shape=(38540,), dtype=object)

### Exportando o DataFrame Completo para ser usado futuramente

In [37]:
df_completo.to_csv('../desafio02/dataset_completo.csv', index = False)